# Creating a Dataframe by Scraping a Wikipedia Page

We will build the  code to scrape following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

## This is the Start of Part 1

In [2]:
# first I will import libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import io

In [3]:
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wiki = requests.get(wikipedia_link).text

# using beautiful soup to parse the result
soup = BeautifulSoup(raw_wiki,'xml')
table = soup.find('table')
df = pd.read_html(str(table), header = 0)[0]
# print(df)

We will ignore cell with unassigned  borough. More than one neighborhood can exist in one postal code area. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [4]:
df = df[df.Borough !=  'Not assigned']
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
# print(df)

Now, let's construct the dataframe

In [5]:
df.to_csv('toronto_wiki.csv')
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
df.shape

(103, 3)

## This is the start of Part 2

In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

In [7]:
data_link = "http://cocl.us/Geospatial_data"
data_cont = requests.get(data_link).content
df2 = pd.read_csv(io.StringIO(data_cont.decode('utf-8')))
df2.columns = ['Postcode', 'Latitude', 'Longitude']
df = pd.merge(df2, df, on = 'Postcode')
df = df[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# This is Start of Part 3

In [8]:
from geopy.geocoders import Nominatim
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [9]:
address = 'Toronto'

geolocator = Nominatim(user_agent="trt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.653963, -79.387207.


In [10]:
# @hiddel_cell
CLIENT_ID = ''# your Foursquare ID, I have removed this intentionally
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20200206' # Foursquare API version

I will only focus on a subset of all the Borough that contain the word  "Toronto".

In [11]:
toronto_data= df[df['Borough'].str.contains('Toronto', na = False)].reset_index(drop=True)
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Let me create a map of Toronto with Boroughs that contain the word "Toronto"

In [13]:
# create map of TORONTO using latitude and longitude values above:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [14]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

Check how many vanue are in each  neighborhood.

In [16]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place, Parkdale Village",21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",18,18,18,18,18,18
"Cabbagetown, St. James Town",45,45,45,45,45,45
Central Bay Street,82,82,82,82,82,82
"Chinatown, Grange Park, Kensington Market",81,81,81,81,81,81
Christie,18,18,18,18,18,18


Let's analyze the feature of each neighborhood 

In [17]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols=list(toronto_onehot.columns.values)
cols.pop(cols.index('Neighborhood'))
toronto_onehot=toronto_onehot[['Neighborhood']+cols]

# rename Neighborhood for Neighbourhood so that future merge works
toronto_onehot.rename(columns = {'Neighborhood': 'Neighbourhood'}, inplace = True)
toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

In [19]:
num_top_venues = 10

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0       Coffee Shop  0.07
1   Thai Restaurant  0.04
2        Steakhouse  0.04
3               Bar  0.04
4              Café  0.04
5    Cosmetics Shop  0.03
6      Burger Joint  0.03
7        Restaurant  0.03
8            Bakery  0.03
9  Asian Restaurant  0.03


----Berczy Park----
                     venue  freq
0              Coffee Shop  0.09
1              Cheese Shop  0.04
2               Steakhouse  0.04
3                   Bakery  0.04
4           Farmers Market  0.04
5                 Beer Bar  0.04
6                     Café  0.04
7             Cocktail Bar  0.04
8       Seafood Restaurant  0.04
9  Comfort Food Restaurant  0.02


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0                    Café  0.14
1          Breakfast Spot  0.10
2             Coffee Shop  0.10
3                  Bakery  0.05
4            Intersection  0.05
5                 Stadium  0.05
6  Furniture / Ho

                      venue  freq
0                Playground  0.33
1                      Park  0.33
2                     Trail  0.33
3         Afghan Restaurant  0.00
4             Moving Target  0.00
5                    Market  0.00
6  Mediterranean Restaurant  0.00
7               Men's Store  0.00
8             Metro Station  0.00
9        Mexican Restaurant  0.00


----North Toronto West----
                     venue  freq
0           Clothing Store  0.12
1              Coffee Shop  0.08
2     Gym / Fitness Center  0.04
3  Health & Beauty Service  0.04
4           Cosmetics Shop  0.04
5             Dessert Shop  0.04
6                    Diner  0.04
7     Fast Food Restaurant  0.04
8                Gift Shop  0.04
9            Grocery Store  0.04


----Parkdale, Roncesvalles----
                         venue  freq
0                    Gift Shop  0.15
1                    Bookstore  0.08
2                  Coffee Shop  0.08
3                Movie Theater  0.08
4  Eastern Europ

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Bar,Steakhouse,Café,Thai Restaurant,Burger Joint,Cosmetics Shop,Bakery,Restaurant,Asian Restaurant
1,Berczy Park,Coffee Shop,Steakhouse,Café,Farmers Market,Seafood Restaurant,Beer Bar,Cocktail Bar,Bakery,Cheese Shop,Greek Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Café,Breakfast Spot,Coffee Shop,Furniture / Home Store,Burrito Place,Restaurant,Italian Restaurant,Stadium,Intersection,Bar
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Spa,Auto Workshop,Brewery,Burrito Place,Comic Shop,Farmers Market,Fast Food Restaurant,Garden,Garden Center
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Plane,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Harbor / Marina,Bar
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Italian Restaurant,Bakery,Pet Store,Pizza Place,Café,Pub,Gym / Fitness Center,Diner
6,Central Bay Street,Coffee Shop,Italian Restaurant,Juice Bar,Japanese Restaurant,Ice Cream Shop,Sandwich Place,Burger Joint,Café,Gym / Fitness Center,Department Store
7,"Chinatown, Grange Park, Kensington Market",Bar,Café,Chinese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Vietnamese Restaurant,Bakery,Burger Joint
8,Christie,Grocery Store,Café,Park,Gas Station,Restaurant,Baby Store,Diner,Italian Restaurant,Athletics & Sports,Candy Store
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Fast Food Restaurant,Gym,Hotel,Café,Gastropub


Last but not least, we shall perform k-means clustering

In [22]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [23]:
toronto_merged = toronto_data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Pub,Health Food Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Ice Cream Shop,Furniture / Home Store,Bookstore,Dessert Shop,Diner,Caribbean Restaurant,Yoga Studio
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Fast Food Restaurant,Park,Sushi Restaurant,Liquor Store,Sandwich Place,Board Shop,Burger Joint,Burrito Place,Fish & Chips Shop,Italian Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Italian Restaurant,Gastropub,Yoga Studio,Fish Market,Pet Store
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Lawyer,Bus Line,Swim School,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [24]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now, let's examine each cluster and understand why are they clustered together. We can see that most are in the first cluster.

In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Trail,Pub,Health Food Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio
1,East Toronto,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Ice Cream Shop,Furniture / Home Store,Bookstore,Dessert Shop,Diner,Caribbean Restaurant,Yoga Studio
2,East Toronto,0,Fast Food Restaurant,Park,Sushi Restaurant,Liquor Store,Sandwich Place,Board Shop,Burger Joint,Burrito Place,Fish & Chips Shop,Italian Restaurant
3,East Toronto,0,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Italian Restaurant,Gastropub,Yoga Studio,Fish Market,Pet Store
4,Central Toronto,0,Park,Lawyer,Bus Line,Swim School,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
5,Central Toronto,0,Gym,Sandwich Place,Park,Department Store,Breakfast Spot,Asian Restaurant,Hotel,Food & Drink Shop,Dog Run,Doner Restaurant
6,Central Toronto,0,Clothing Store,Coffee Shop,Yoga Studio,Grocery Store,Salon / Barbershop,Restaurant,Rental Car Location,Park,Mexican Restaurant,Metro Station
7,Central Toronto,0,Dessert Shop,Sandwich Place,Pizza Place,Gym,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Fried Chicken Joint,Indoor Play Area
8,Central Toronto,0,Park,Playground,Trail,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
9,Central Toronto,0,Coffee Shop,Pub,Light Rail Station,American Restaurant,Sushi Restaurant,Fried Chicken Joint,Sports Bar,Restaurant,Pizza Place,Liquor Store


In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Central Toronto,1,Sandwich Place,Café,Coffee Shop,Pizza Place,Liquor Store,Burger Joint,Middle Eastern Restaurant,Indian Restaurant,Pub,History Museum
28,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Seafood Restaurant,Italian Restaurant,Hotel,Japanese Restaurant,Beer Bar,Park,Cheese Shop


In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Steakhouse,Asian Restaurant,Japanese Restaurant,Gastropub,Seafood Restaurant,Bar,Gym
